Problem https://projecteuler.net/problem=954

# Main idea

Generate states for each number, the states can construct a following state adding a new number to the left.

For each number we store the module, the digits and its multipliers ($10^n mod 7$) in module 7 and also the possible results of the modulo 7 after one permutation of digits.

The 0 thing complicate a little bit the things, but with some extra variables it is possible.

The states doesn't improve much, because there are a lot of states, so my solution is very slow.



In [1]:
def get_complete_state(new_digit: int):
    """Return a state that joins all the completed numbers"""
    return (
            0,
            tuple(range(7)),
            (),
            new_digit == 0,
            (),
            (),
        )

def transform_state(state, new_digit, coef):
    if len(state[1]) == 7:
        # This case will allow any new number and will avoid generating different states
        return get_complete_state(new_digit)
    add_to_mod = (coef * new_digit) % 7
    number_mod_7 = (state[0] + add_to_mod) % 7

    values_after_one_swap = {
        *((x + add_to_mod) % 7 for x in state[1]),
        *((x + add_to_mod) % 7 for x in state[2])
    } # add the new digit to the options after swapping another 2 digits. Now we can use swaps that previosuly resulted in trailing 0
    for digit, coef_1 in state[4]:
        if new_digit != digit and coef != coef_1:
            values_after_one_swap.add((number_mod_7 - (new_digit - digit) * (coef - coef_1)) % 7)

    values_with_swaps_with_0 = set()
    for digit, coef_1 in state[5]:
        # Maybe improve this code to avoid duplicates, but is a small improve
        values_with_swaps_with_0.add((number_mod_7 - (new_digit - digit) * (coef - coef_1)) % 7)
    options_to_move = state[4]
    options_to_move_with_0 = state[5]
    
    if new_digit == 0:
        options_to_move_with_0 = tuple(sorted({(0, coef), *options_to_move_with_0}))
    else:
        options_to_move = tuple(sorted({(new_digit % 7, coef), *options_to_move}))
    if len(values_after_one_swap) == 7:
        return get_complete_state(new_digit)
    return (
        number_mod_7,
        tuple(sorted(values_after_one_swap)),
        tuple(sorted(values_with_swaps_with_0)),
        new_digit == 0,
        options_to_move,
        options_to_move_with_0
    )
    

In [ ]:
%%prun -s cumulative

from collections import Counter


states = {
    (
        e % 7, # number mod 7
        (e % 7,), # possible values with 1 swap
        (), # possible values with swapping 0 at the first element
        e == 0, # has trailing 0
        ((e % 7, 1),) if e else (), # (options_to_move in the form: (digit, 10^n mod 7))
        () if e else ((0, 1),),
    ): 1 if e <= 7 else 2
    for e in range(10)
}

heptaphobics = 8
for n in range(1, 13):
    coef = (10 ** n) % 7
    new_states = Counter()
    for new_digit in range(8):
        mult = 1 if new_digit not in (1, 2) else 2
        for state, count in states.items():
            new_states[transform_state(state, new_digit, coef)] += count * mult
    heptaphobics += sum(count for state, count in new_states.items() if 0 != state[1][0] and not state[3])
    states = new_states
    print(f'{n}: {heptaphobics} with {len(states)} states')

    

1: 74 with 64 states
2: 573 with 512 states
3: 3737 with 4096 states
4: 20714 with 27148 states
5: 96202 with 123558 states
6: 423191 with 426538 states
